In [1]:
! python3 --version

Python 3.12.3


In [ ]:
! pip install transformers torch accelerate

In [3]:
! pip freeze > requirements.txt

In [ ]:
# Do this from terminal so you can paste the token when requested
! huggingface-cli login

In [6]:
! huggingface-cli whoami

Gilgamesh37
